___
# Ciência dos Dados - PROJETO 1

## Gabriel Couto & Daniel

## Perfil dos brasileiros economicamente ativos

___
## Introdução

#Botar UF <br>

As variáveis selecionadas: <br>
    -Idade <br>
    -Sexo <br>
    -Estado Civil <br>
    -Economicamente Ativo ou não <br>
    -CH de trabalho (h) <br>
    -Nível acadêmico mais elevado que alcançou <br>
    -Rendimento mensal de todos os trabalhos (reais) <br>
    -Rendimento mensal de todas as fontes(reais) <br>

Analizar
Descreva aqui o que pretende analisar!

Capriche no português!! 
Se quiser pode utilizar pesquisas semelhantes publicadas em sites confiáveis!!


___
## Filtros da base de dados

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
tabela = pd.read_table("PES2015.csv", encoding="windows-1252", sep=",")

In [18]:
colunas = ['V8005', #Int
           'V0302','V4011','V4704','V4707','V6007', #Category
           'V4719','V4720'] #Float

dados = tabela.loc[:,colunas]

#Adotando nomes mais identificáveis às colunas:
colunas[0] = 'Idade'

colunas[1] = 'Sexo'
colunas[2] = 'Estado Civil' 
colunas[3] = 'Economicamente' #Condição de atividade na semana
colunas[4] = 'CH de trabalho (h)' #Horas trabalhadas na semana em todos empregos (Carga Horária de Trabalho semanal)
colunas[5] = 'Maior graduação' #Curso mais elevado que frequentou

colunas[6] = 'Rend. trabalho(R$)' #Rendimento mensal de todos os trabalhos (para maiores de 10 anos)
colunas[7] = 'Rend. total(R$)' #Rendimento mensal de todas fontes (para maiores de 10 anos)

#Renomeando as colunas:
dados.columns = colunas


#Definindo as variáveis qualitativas como categóricas:
dados[colunas[1]] = dados[colunas[1]].astype('category')
dados[colunas[2]] = dados[colunas[2]].astype('category')
dados[colunas[3]] = dados[colunas[3]].astype('category')
dados[colunas[4]] = dados[colunas[4]].astype('category') 
dados[colunas[5]] = dados[colunas[5]].astype('category')

#Definindo as categorias das variáveis categóricas:
dados[colunas[1]].cat.categories=(['Masculino','Feminino'])
dados[colunas[2]].cat.categories=(['Solteiro','Casado','Desquisado','Divorciado','Viúvo'])
dados[colunas[3]].cat.categories=(['Ativa','Não Ativa'])
dados[colunas[4]].cat.categories=(['< 14','15 a 39','40 a 44','45 a 48','> 49'])
dados[colunas[5]].cat.categories=(['Elementar','Médio 1º ciclo','Médio 2º ciclo','Regular EF','Regular EM','Supletivo EF','Supletivo EM','Graduação','Mestrado ou Doutorado','Supletivo de Alfabetização','Creche','Alfabetização','Maternal']) #EF = Ensino Fundamental | EM  = Ensino Médio

In [17]:
ativos = dados
dados.head(20)

,Idade,Sexo,Estado Civil,Economicamente,CH de trabalho (h),Maior graduação,Rend. trabalho(R$),Rend. total(R$)
0,23,Masculino,Solteiro,Ativa,40 a 44,Regular EM,800.0,800.0
1,23,Feminino,Divorciado,Ativa,40 a 44,Regular EM,1150.0,1150.0
2,35,Feminino,NaN,Ativa,40 a 44,NaN,880.0,880.0
3,34,Masculino,NaN,Ativa,> 49,Graduação,3000.0,3000.0
4,11,Feminino,Solteiro,Não Ativa,NaN,NaN,NaN,0.0
5,7,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
6,4,Feminino,NaN,NaN,NaN,NaN,NaN,NaN
7,18,Feminino,Solteiro,Ativa,40 a 44,Regular EM,880.0,880.0
8,46,Masculino,Solteiro,Ativa,> 49,Regular EF,3500.0,3500.0
9,81,Feminino,Viúvo,Não Ativa,NaN,Elementar,NaN,2300.0


___
## Análise e Resultados

___
## Conclusão 

Descreva aqui suas conclusões!